# BioSTEAM User Guide Tutorial - Beginner's Guide
this runs in Ubuntu 22.04.2 LTS on WSL2, Python 3.10.12\
install BioSTEAM with 'pip install biosteam'\
see https://biosteam.readthedocs.io/en/latest/tutorial/index.html

## 1. Getting Started
see https://biosteam.readthedocs.io/en/latest/tutorial/Getting_started.html

### Create a Stream object

In [2]:
import biosteam as bst
from biosteam import settings
bst.nbtutorial() # Light-mode html diagrams and filter warnings
settings.set_thermo(['Water', 'Methanol'])
# set flow rates in kmol/h
feed = bst.Stream(Water=50, Methanol=20)
feed.show()

Stream: s1
phase: 'l', T: 298.15 K, P: 101325 Pa
flow (kmol/hr): Water     50
                Methanol  20


Set prices for performing techno-economic analysis later:

In [3]:
feed.price = 0.15 # USD/kg
feed.cost # USD/hr

231.24018

Set the chemical engineering plant cost index:

In [4]:
settings.CEPCI # Default year is 2017
settings.electricity_price = 0.065 # Adjust price

Set HeatUtility options via UtilityAgent objects (Stream objects with additional attributes):

In [5]:
# show all available cooling agents
settings.cooling_agents
cooling_water = settings.get_cooling_agent('cooling_water')
# show the UtilityAgent
cooling_water.show()
# set price of regenerating the utility in USD/kmol
cooling_water.regeneration_price

UtilityAgent: cooling_water
heat_transfer_efficiency: 1.000
heat_transfer_price: 0 USD/kJ
regeneration_price: 0.000488 USD/kmol
T_limit: 325 K
phase: 'l'
T: 305.37 K
P: 101325 Pa
flow (kmol/hr): Water  1


0.00048785

Find design requirements and cost with Unit objects:

In [6]:
# create a Flash object
# Specify vapor fraction and isobaric conditions
F1 = bst.Flash('F1', ins=feed, V=0.1, P=101325)
F1.show()
# Note that the outlets, outs, is populated by empty streams.

Flash: F1
ins...
[0] s1  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] s3  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [7]:
# To simulate the flash, use the simulate method:
F1.simulate()
F1.show()

Flash: F1
ins...
[0] s1  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2  
    phase: 'g', T: 352.84 K, P: 101325 Pa
    flow (kmol/hr): Water     2.59
                    Methanol  4.41
[1] s3  
    phase: 'l', T: 352.84 K, P: 101325 Pa
    flow (kmol/hr): Water     47.4
                    Methanol  15.6


In [22]:
# The results method returns simulation results:
display(
    F1.results(), # Default returns DataFrame object with units
    F1.net_duty, # Duty with heat transfer losses [kJ / hr]
    F1.net_power, # Electricity consumption [kW]
    [F1.feed, F1.vapor, F1.liquid] # Inlet feed and vapor and liquid outlets
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2591885501.py, line 3)

Design a chemical process consisting of a flash with a partial liquid recycle:

In [8]:
recycle = bst.Stream('liquid_recycle')
feed = bst.Stream('feed', Methanol=100, Water=450)
M1 = bst.Mixer('M1', ins=(recycle, feed))
F1 = bst.Flash('F1',
    ins=M1-0, # -pipe- notation equivalent to M1.outs[0]
    outs=('vapor_product', 'liquid'),
    V=0.1, P=101325
)
S1 = bst.Splitter('S1',
    ins=F1-1, # -pipe- notation equivalent to F1.outs[1]
    outs=(recycle, 'liquid_product'),
    split=0.5 # Split to 0th output stream
)
# find unit operations and manage flowsheets
bst.main_flowsheet.diagram()
# create a system using the flowsheet:
sys = bst.main_flowsheet.create_system('sys')
sys.show()

System: sys
ins...
[0] feed  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] liquid_product  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [9]:
# simulate
sys.simulate()
sys.show()

System: sys
Highest convergence error among components in recycle
stream S1-0 after 4 loops:
- flow rate   3.12e-01 kmol/hr (0.57%)
- temperature 3.56e-02 K (0.0099%)
ins...
[0] feed  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product  
    phase: 'g', T: 359.26 K, P: 101325 Pa
    flow (kmol/hr): Water     53.4
                    Methanol  46.7
[1] liquid_product  
    phase: 'l', T: 359.26 K, P: 101325 Pa
    flow (kmol/hr): Water     397
                    Methanol  53.6


Show system-level results

In [10]:
sys.results()

System                           Units      sys
Low pressure steam       Duty    kJ/hr 6.76e+06
                         Flow  kmol/hr      175
                         Cost   USD/hr     41.5
Total purchase cost                USD 3.64e+04
Installed equipment cost           USD 1.08e+05
Utility cost                    USD/hr     41.5
Material cost                   USD/hr        0
Sales                           USD/hr        0

In [11]:
# this requires a graphical environment to create the flowchart.png file
sys.save_report('Example.xlsx') # save report
# Note that the cash flow analysis requires a TEA object with all the necessary parameters